In [42]:
import codecs
import pandas as pd
import glob
import re
import sys
import time
path="./paper_data/B"
place_catch="常滑"
data=pd.DataFrame(columns=["date","place","race_num","race_rank","position","id","name","age","belongs","weight","player_rank","country_win","country_sub","local_win","local_sub","motor","boat"])

In [43]:
def data_clean(data):
    tmp=data.replace("\u3000","")
    tmp=tmp.replace("\r\n","")
    tmp=re.split(" +",tmp)
    return tmp

In [44]:
def get_place(tmp):
    place=data_clean(tmp)
    return place


def add_data(date,test,each,weather,place):
    st=re.split(" +",each)
    name=st[1].replace("\u3000"," ")
    tmp_data=[st[0],name[0:4],name[4:8],name[8:10],name[10:12],name[12:14],name[14:16]]+[each[18:23],each[23:29],each[29:34],each[34:40],each[43:49],each[52:58]]
    concat_data=[date,place]+[weather[0][:-1],weather[1]]+tmp_data
    add=pd.DataFrame([concat_data],columns=test.columns)
    
    return add

In [45]:
def get_data(date,test,file,place_catch):
    line_num=0
    tmp=[-1,-1]
    hyphen=0
    with codecs.open(file,"r", "Shift-JIS", "ignore") as race:
        for i,each in enumerate(race):

            #ボートレース開催地を取得
            if each.find("ボートレース")!=-1:
                place_tmp=get_place(each)
                place=place_tmp[-1].replace("ボートレース","")

            #レース情報の前に発生するハイフン列を取得    
            if each.find("---")!=-1:
                hyphen+=1
                if hyphen%2==1:
                    weather=line_even
                if hyphen%2==0:
                    tmp=[i+1,i+6]

            if line_num%2==1:
                line_even=data_clean(each)
            line_num+=1

            # リストにデータを追加
            if i>=tmp[0] and i<=tmp[1] and place==place_catch:
                test=test.append(add_data(date,test,each,weather,place_catch))
    return test

In [46]:
st=time.time()
for file in glob.glob(path+"*.TXT"):
    sys.stdout.write("\r call:{}".format(file))
    tmp_data=pd.DataFrame(columns=data.columns)
    date=file.replace(path,"")
    date=date.replace(".TXT","")
    tmp_data=get_data(date,tmp_data,file,place_catch)
    data=pd.concat([data,tmp_data],axis=0)
print("\nfinish:{}sec".format(time.time()-st))

 call:./paper_data/B180506.TXT
finish:21.42658805847168sec


In [47]:
data.to_csv("csv_files/paper_data.csv",encoding="shift_jis")